In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as imbpipeline

# Lendo os dados

In [ ]:
train = pd.read_csv('dataset.csv')

In [ ]:
train.head()

# Verificando informações sobre os dados

In [ ]:
# Verificando algumas informações sobre o dataset
train.info()

In [ ]:
# Verificando o número de linhas e colunas do dataset
print(f'Número de linhas: {train.shape[0]}')
print(f'Número de colunas: {train.shape[1]}')

In [ ]:
# Verificando se há valores faltantes no dataset
train.isnull().sum()

In [ ]:
# Verificamos que há uma linha com valores nulos
# Removendo a linha nula
train = train.dropna()
train.isnull().sum()

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
train['Churn'].value_counts(normalize=True)

# EDA

In [ ]:
# Removendo a variável "CustomerID"
train = train.drop('CustomerID', axis=1)

In [ ]:
train.head()

## Variáveis categóricas

### Variável 'Churn'

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(x='Churn', data=train)

# Adicionando os valores acima da barra
for p in ax.patches:
    height = p.get_height()
    ax.text(
        x=p.get_x() + p.get_width() / 2,  # posição X central da barra
        y=height + 5,                     # ligeiramente acima da barra
        s=f'{int(height)}',               # valor a ser exibido
        ha='center'                       # alinhamento horizontal
    )

# Adicionando titulo
plt.title('Distribuição da variável "Churn"')
plt.show()

### Variável 'Gender'

In [ ]:
fig = px.histogram(train, x='Gender', text_auto = True, color = 'Churn', barmode = 'group')
# Adiciona o título
fig.update_layout(title="Distribuição de Churn por Gênero")

Visualizando o gráfico, vemos que mulheres têm uma maior taxa de churn em relação ao público masculino

### Variável 'Subscription Type'

In [ ]:
fig = px.histogram(train, x='Subscription Type', text_auto = True, color = 'Churn', barmode = 'group')
# Adiciona o título
fig.update_layout(title="Distribuição de Churn por Tipo de Inscrição")

Pessoas que tem o plano "Standard" tendem a cancelarem mais o serviço.

### Variável 'Contract Length'

In [ ]:
fig = px.histogram(train, x='Contract Length', text_auto = True, color = 'Churn', barmode = 'group')
# Adiciona o título
fig.update_layout(title="Distribuição de Churn por Tipo de Contrato")

- Pessoas com o tipo de contrato "Monthly" têm uma maior taxa de churn em relação aos outros planos.
- É visualizado que no contrato "Monthly" não possui dados para pessoas que não cancelaram o serviço.

## Variáveis numéricas

### Variável 'Age'

In [ ]:
px.box(train, x = 'Age', color = 'Churn')

- É verificado que a média de idade das pessoas que cancelam o serviço é maior das que não cancelam, mesmo que por pouca diferença.

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(train['Age'], bins=15)
plt.title('Distribuição da Idade')
plt.xlabel('Age')
plt.show()

- A idade entre 40 e 43 anos é a que tem maior número de pessoas com o serviço.

### Variável 'Tenure'

In [ ]:
px.box(train, x = 'Tenure', color = 'Churn')

### Variável 'Usage Frequency'

In [ ]:
px.box(train, x = 'Usage Frequency', color = 'Churn')

- É visualizado que pessoas que cancelam o serviço tem uma média de uso menor que as pessoas que utilizam.

### Variável 'Support Calls'

In [ ]:
px.box(train, x = 'Support Calls', color = 'Churn')

- Uma informação importante é que as pessoas que cancelam o serviço realizam, em média, mais ligações de suporte que as pessoas que não cancelam.

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='Churn', y = 'Support Calls', data=train)
plt.title('Support Calls vs Churn')
plt.xlabel('Churn (0 = No, 1 = Yes)')
plt.show()

### Variável 'Payment Delay'

In [ ]:
px.box(train, x = 'Payment Delay', color = 'Churn')

### Variável "Total Spend"

In [ ]:
px.box(train, x = 'Total Spend', color = 'Churn')

# Pré-processamento dos dados

In [ ]:
train.head()

In [ ]:
variavel_categoricas = ['Gender', 'Subscription Type', 'Contract Length']

In [ ]:
# Preprocessamento do dados
encoder = OneHotEncoder(drop='if_binary')

df_categorico = train[variavel_categoricas]

df_encoded = pd.DataFrame(encoder.fit_transform(df_categorico).toarray(),
                         columns=encoder.get_feature_names_out(variavel_categoricas))

# Unindo os dataframes
df_final = pd.concat([train.drop(variavel_categoricas, axis=1), df_encoded], axis=1)

In [ ]:
df_final = df_final.drop(199295)
df_final = df_final.drop(440832)

In [ ]:
df_final[df_final.isnull().any(axis=1)]

# Divisão em Treino e Teste

In [ ]:
df_final.head()

In [ ]:
# Removendo a coluna target do conjunto de treino
X = df_final.drop('Churn', axis=1)
y = df_final['Churn']

In [ ]:
# Utilização no final do projeto
X, X_teste, y, y_teste = train_test_split(X, y, test_size = 0.15, stratify=y, random_state=0)

# Separação em treino e Validação
# X e y utilizado na parte de cima
X_treino, X_val, y_treino, y_val = train_test_split(X, y, stratify=y, random_state=0)

# Treinamento dos Modelos

## Árvore de Decisão

### Treinando o modelo de árvore de decisão

In [ ]:
nome_colunas = [
    'Male',
    'Basic',
    'Premium',
    'Standard',
    'Annual',
    'Monthly',
    'Quarterly',
    'Age',
    'Tenure',
    'Usage Frequency',
    'Support Calls',
    'Payment Delay',
    'Total Spend',
    'Last Interaction']

In [ ]:
arvore = DecisionTreeClassifier(max_depth=3, random_state=5)
arvore.fit(X_treino, y_treino)

# Fazendo previsoes
y_pred = arvore.predict(X_val)

print(f'Acurácia de treino: {arvore.score(X_treino, y_treino)}')
print(f'Acurácia de validação: {arvore.score(X_val, y_val)}')

In [ ]:
# Plotando a árvore
plt.figure(figsize=(15,6))
plot_tree(arvore, filled = True, class_names = ['não', 'sim'], fontsize = 7, feature_names = nome_colunas);

### Avaliando o desempenho do modelo

In [ ]:
print(f'Acurácia do modelo: {(arvore.score(X_teste, y_teste))}')

In [ ]:
matriz_confusao = confusion_matrix(y_val, y_pred)

In [ ]:
visualizacao = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao, display_labels=arvore.classes_)
visualizacao.plot();

In [ ]:
# Melhorando a visualização da matriz de confusão
visualizacao = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao, display_labels = ['Não Churn', 'Churn'])
visualizacao.plot();

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val, y_pred, normalize='true', cmap = 'Blues');

### Relatório de métricas

In [ ]:
print(classification_report(y_val, y_pred))

## Random Forest

In [ ]:
modelo_rf = RandomForestClassifier(random_state=0)
modelo_rf.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = modelo_rf.predict(X_val)

print(f'Acurácia de treino: {modelo_rf.score(X_treino, y_treino)}')
print(f'Acurácia de validação: {modelo_rf.score(X_val, y_val)}')

In [ ]:
# Ajustando alguns hiperparâmetros
modelo_rf = RandomForestClassifier(random_state=0, max_depth=10)
modelo_rf.fit(X_treino, y_treino)
# Fazendo previsões
y_pred = modelo_rf.predict(X_val)

print(f'Acurácia de treino: {modelo_rf.score(X_treino, y_treino)}')
print(f'Acurácia de validação: {modelo_rf.score(X_val, y_val)}')

### Avaliando o modelo

In [ ]:
matriz_confusao = confusion_matrix(y_val, y_pred)

In [ ]:
visualizacao = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao)
visualizacao.plot();

### Relatório de métricas

In [ ]:
report = classification_report(y_val, y_pred)
print('Relatório de Classificação:\n', report)

# Balanceando os dados

In [ ]:
y_treino.value_counts()

In [ ]:
oversample = SMOTE(random_state=0)

# Balanceamento dos dados na base de treino
X_balanceado, y_balanceado = oversample.fit_resample(X_treino, y_treino)

In [ ]:
y_balanceado.value_counts()

In [ ]:
# Treinando o modelo de Random Forest com os dados balanceados
modelo = RandomForestClassifier(random_state=0, max_depth=10)
modelo.fit(X_balanceado, y_balanceado)
y_pred = modelo.predict(X_val)

In [ ]:
print(classification_report(y_val, y_pred))
ConfusionMatrixDisplay.from_predictions(y_val, y_pred, normalize='true', cmap='Blues');

# Realizando a validação cruzada

In [ ]:
modelo = RandomForestClassifier(max_depth=10)
pipeline = imbpipeline([('oversample', SMOTE()), ('floresta', modelo)]) # Etapas do pipeline

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cv_resultados = cross_validate(pipeline, X, y, cv=skf, scoring='recall_weighted')
cv_resultados['test_score']

In [ ]:
# Criando um intervalo de confiança
media = cv_resultados['test_score'].mean()
desvio_padrao = cv_resultados['test_score'].std()
print(f'Intervalo de confiança: [{media - 2 * desvio_padrao}, {min(media + 2 * desvio_padrao, 1.0)}]')

# Usando dados de teste

In [ ]:
oversample = SMOTE(random_state=0)
X_balanceado, y_balanceado = oversample.fit_resample(X, y)
modelo = RandomForestClassifier(random_state=0, max_depth=10)
modelo.fit(X_balanceado, y_balanceado)
y_pred = modelo.predict(X_teste)

print(classification_report(y_teste, y_pred))
ConfusionMatrixDisplay.from_predictions(y_teste, y_pred, normalize='true', cmap='Blues');

# Conclusão

In [ ]:
print(f'Acurácia do modelo de árvore: {arvore.score(X_val, y_val)}')
print(f'Acurácia do modelo de RandomForest: {modelo_rf.score(X_val, y_val)}')